Zalecamy nie czytać notatników na githubie, ze względu na źle wyświetlające się wizualizacje i brak możliwości uruchamiania kodu. Polecamy otworzyć notatnik w google colab, następującym linkiem:

<a target="_blank" href="https://colab.research.google.com/github/asia281/StaszicAI/blob/main/04-06/Mnist_pytorch.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

Code based on https://github.com/pytorch/examples/blob/master/mnist/main.py

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import plotly.graph_objects as go
import sys

In [ ]:
# TODO: create model. Hint: use input size: 28*28,
# hidden sizes: 128, 128, output size: 10, and ReLU.

model = ...

In [ ]:
def train(
    model,
    device,
    train_loader,
    optimizer,
    epoch,
    log_interval,
    visualizer,
):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        data = torch.flatten(data, 1)  # Flatten the input
        optimizer.zero_grad()
        # Pass data through model
        ...
        output = F.log_softmax(output)  # And compute probabilities for classes
        loss = F.nll_loss(output, target)  # As well as NLL loss
        # TODO: add backward step for loss
        ...
        visualizer.plot_gradients_and_loss(batch_idx, loss.item())
        # TODO: add optimizer step
        ...
        # TODO: print loss for every log_interval step
        ...

In [ ]:
def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            # TODO: flatten input, pass it through model and log_softmax
            ...
            ...
            output = ...
            test_loss += F.nll_loss(
                output, target, reduction="sum"
            ).item()  # sum up batch loss
            pred = output.argmax(
                dim=1, keepdim=True
            )  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    # TODO: print accuracy and loss
    ...

In [ ]:
# Defining hyperparams
batch_size = 256
test_batch_size = 1000
epochs = 5
seed = 1
log_interval = 10
use_cuda = torch.cuda.is_available()

In [ ]:
# Loading datasets
torch.manual_seed(seed)
device = torch.device("cuda" if use_cuda else "cpu")

train_kwargs = {"batch_size": batch_size}
test_kwargs = {"batch_size": test_batch_size}
if use_cuda:
    cuda_kwargs = {"num_workers": 1, "pin_memory": True, "shuffle": True}
    train_kwargs.update(cuda_kwargs)
    test_kwargs.update(cuda_kwargs)

transform = transforms.Compose(
    [transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))]
)
dataset1 = datasets.MNIST("../data", train=True, download=True, transform=transform)
dataset2 = datasets.MNIST("../data", train=False, transform=transform)
train_loader = torch.utils.data.DataLoader(dataset1, **train_kwargs)
test_loader = torch.utils.data.DataLoader(dataset2, **test_kwargs)

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ../data/MNIST/raw/train-images-idx3-ubyte.gz to ../data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../data/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../data/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../data/MNIST/raw



In [ ]:
model = model.to(device)

# TODO: Define learning rate and optimizer
lr = ...
optimizer = ...

# Loop for number of epochs
for epoch in range(1, epochs + 1):
    # TODO: run train
    ...
    # TODO: run test
    ...

In [ ]:
# Check results visually
import matplotlib.pyplot as plt

model.eval()
with torch.no_grad():
    data, target = next(iter(test_loader))
    data, target = data.to(device), target.to(device)
    data = torch.flatten(data, 1)
    output = model(data)
    output = F.log_softmax(output).detach().cpu()
    num_row = 2
    num_col = 5

    # plot images
    fig, axes = plt.subplots(num_row, num_col, figsize=(1.5 * num_col, 2 * num_row))
    for i in range(10):
        ax = axes[i // num_col, i % num_col]
        ax.imshow(data[i].reshape(28, 28), cmap="gray")
        ax.set_title("Predicted: {}".format(output.argmax(dim=1, keepdim=True)[i][0]))
    plt.tight_layout()
    plt.show()